In [1]:
import AdventOfCode
import Data.List.Split
import qualified Data.Attoparsec.Char8 as A

In [2]:
raw <- splitOn "\n\n" <$> dayString 4

In [3]:
import Data.List (sort, find)
import Data.Maybe (isJust)

processed = map (sort . words . unwords . lines) raw

In [4]:
import qualified Data.Set as Set

In [5]:
attrsOnly = map (Set.fromList . map (take 3)) processed

allRequired attrs = let
    required = Set.fromList    
        [ "byr"
        , "iyr"
        , "eyr"
        , "hgt"
        , "hcl"
        , "ecl"
        , "pid"
        ]
    in Set.intersection required attrs == required
length $ filter allRequired attrsOnly

254

In [6]:
zipped = zip attrsOnly processed
valid = map snd $ filter (allRequired . fst) zipped

In [7]:
import qualified Data.ByteString.Char8 as BS

unvalidated = map (map BS.pack) valid

In [8]:
{-# LANGUAGE OverloadedStrings #-}

data Passport = Passport
    { byr
    , iyr
    , eyr
    , hgt
    , hcl
    , ecl
    , pid :: String
    } deriving (Eq, Show)

parsePassport :: [String] -> Passport
parsePassport attrs = Passport
    (maybe "" (drop 4) $ find (\f -> take 4 f == "byr:") attrs)
    (maybe "" (drop 4) $ find (\f -> take 4 f == "iyr:") attrs)
    (maybe "" (drop 4) $ find (\f -> take 4 f == "eyr:") attrs)
    (maybe "" (drop 4) $ find (\f -> take 4 f == "hgt:") attrs)
    (maybe "" (drop 4) $ find (\f -> take 4 f == "hcl:") attrs)
    (maybe "" (drop 4) $ find (\f -> take 4 f == "ecl:") attrs)
    (maybe "" (drop 4) $ find (\f -> take 4 f == "pid:") attrs)

In [9]:
passports = map parsePassport valid

In [10]:
import qualified Text.ParserCombinators.ReadP as P
import Text.Read (readMaybe)

hexDigit = P.choice $ map P.char $ ['0'..'9'] <> ['a'..'f']
digit = P.choice $ map P.char ['0'..'9']
failed = pure False

validateByr = do
    yrS <- P.count 4 digit
    let yr = maybe 0 id (readMaybe yrS :: Maybe Int)
    P.eof
    pure $ 1920 <= yr && yr <= 2002
    
validateIyr = do
    yrS <- P.count 4 digit
    let yr = maybe 0 id (readMaybe yrS :: Maybe Int)
    P.eof
    pure $ 2010 <= yr && yr <= 2020

validateEyr = do
    yrS <- P.count 4 digit
    let yr = maybe 0 id (readMaybe yrS :: Maybe Int)
    P.eof
    pure $ 2020 <= yr && yr <= 2030

validateHgt = do
    hS <- P.many1 digit
    let h = maybe 0 id (readMaybe hS :: Maybe Int)
    unit <- P.choice $ map P.string ["cm", "in"]
    P.eof
    pure $ case unit of
        "cm" -> 150 <= h && h <= 193
        "in" -> 59 <= h && h <= 76

validateHcl = do
    P.char '#'
    P.count 6 hexDigit
    P.eof
    pure True

validateEcl = do
    P.choice $ map P.string ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
    P.eof
    pure True

validatePid = do
    P.count 9 digit
    P.eof
    pure True

Line 10: Use fromMaybe
Found:
maybe 0 id
Why not:
Data.Maybe.fromMaybe 0Line 16: Use fromMaybe
Found:
maybe 0 id
Why not:
Data.Maybe.fromMaybe 0Line 22: Use fromMaybe
Found:
maybe 0 id
Why not:
Data.Maybe.fromMaybe 0Line 28: Use fromMaybe
Found:
maybe 0 id
Why not:
Data.Maybe.fromMaybe 0

In [11]:
import Control.Applicative ((<|>))

validatePassport p = and
        [ fst . head $ P.readP_to_S (validateByr P.<++ failed) (byr p)
        , fst . head $ P.readP_to_S (validateIyr P.<++ failed) (iyr p)
        , fst . head $ P.readP_to_S (validateEyr P.<++ failed) (eyr p)
        , fst . head $ P.readP_to_S (validateHgt P.<++ failed) (hgt p)
        , fst . head $ P.readP_to_S (validateHcl P.<++ failed) (hcl p)
        , fst . head $ P.readP_to_S (validateEcl P.<++ failed) (ecl p)
        , fst . head $ P.readP_to_S (validatePid P.<++ failed) (pid p)
        ]

In [12]:
length $ filter id $ map validatePassport passports

184